## Finite state space example

现在在我们已知 $(M_t, I_t, S_t)$, 要估计fair price。为了对离散过程进行建模同时减少计算量，我们进行以下处理：

1. 我们把 $I_t$ 分为 n 段，每一段的表示为如下公式。比如说[1，2，3，4，5]分别表示imbalance程度为[0-0.2, 0.2-0.4, 0.4-0.6, 0.6-0.8, 0.8-1.0]：
$$
I_t=\sum_{j=1}^nj\mathbb{1}_{\left(\frac{j-1}{n}<\frac{Q_t^b}{Q_t^b+Q_t^a}\leq\frac{j}{n}\right)}
$$

2. 同时 spread 的值为离散值，服从 1<= s <= m

所以状态 $(I_t, S_t)$ 为离散值，且取值有 nm 个。

3. 我们使用$K=\begin{bmatrix}-0.01,-0.005,0.005,0.01\end{bmatrix}^T$来表示 mid-price 的变动，（或者取值为 -1个tick，-0.5个tick， 0.5个tick， 1个tick）

则对于下一个时刻的 mid-price 的变动来说，服从以下的公式：

$$\begin{aligned}
G^{1}(x)& =\mathbb{E}\left[M_{\tau_{1}}-M_{t}|X_{t}=x\right]  \\
&= \sum_{k\in K}k\cdot\mathbb{P}(M_{\tau_{1}}-M_{t}=k|X_{t}=x)  \\
&=\sum_{k\in K}\sum_{u}k\cdot\mathbb{P}(M_{\tau_{1}}-M_{t}=k\wedge\tau_{1}-t=u|X_{t}=x)
\end{aligned}$$

我们估计两种状态：
1. R := absorbing states, 可以理解为在给定 $(I_t, S_t)$ 下，mid-price发生改变k的概率，其中k是上文中定义的K的取值。所以矩阵的维度是 4 x nm
$$R_{xk}:=\mathbb{P}(M_{t+1}-M_t=k|X_t=x)$$


2. Q :=  transient states, 可以理解为在给定 $x = (I_t, S_t)$ 下，mid-price 不发生改变且下一个状态是新的$y = (I_t, S_t)$ 的概率。所以矩阵的维度是 nm x nm
$$Q_{xy}:=\mathbb{P}(M_{t+1}-M_t=0\wedge X_{t+1}=y|X_t=x)$$

所以下一个时刻，mid-price 发生改变的期望是：
$$G^1(x)=\bigl(\sum_sQ^{s-1}R\bigr)K=\bigl(1-Q\bigr)^{-1}RK$$

通过递归我们就可以算出来$G^{i+1}(x)$


### 最终公式

为了方便计算，我们重新定义 absorbing states T, 新的矩阵维度是 nm x nm:
$$T_{xy}:=\mathbb{P}(M_{t+1}-M_t\neq0\wedge X_{t+1}=y|X_t=x)$$

定义$B:=\left(1-Q\right)^{-1}T$， B 显然是一个 nm x nm 的矩阵。则最终的价格为：
$$P_t^i=M_t+\sum_{k=0}^iB^kG^1$$

THEOREM $3.1\quad If\:B^*=\lim_{k\to\infty}B^k\:and\:B^*G^1=0,\:then\:the\:limit$
$\lim_{i\to\infty}P_{t}^{i}=P_{t}^{micro}$
$converges.$

The matrix $B$ is a regular stochastic matrix so it can be decomposed

$$
B=B^*+\sum_{j=2}^{nm}\lambda_jB_j
$$

所以最终的公式为：

$$P_t^{micro}=\lim\limits_{i\to\infty}P_t^i=M_t+G^1+\sum\limits_{j=2}^{nm}\frac{\lambda_j}{1-\lambda_j}B_jG^1$$



# 计算步骤
1. symmetrize data
    - (It,St,It+1,St+1,dM) ==> (1-It,St,1-It+1,St+1,-dM)
2. Estimate Q, T, R
3. Calculate G1
4. Calculate G* ==> micro price adjustment 


In [1]:
# 这个库可以显著地加速运算
!conda install scikit-learn-intelex -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0



# All requested packages already installed.



In [2]:
import os, sys, datetime, logging
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
import plotly.express as px
from datetime import timedelta
import joblib

if '../' not in sys.path:
    sys.path.append('../')
if 'load_s3/' not in sys.path:
    sys.path.append('load_s3/')
    
import import_ipynb
from util.s3_method import * 
from util.time_method import *
from util.plot_method import easy_plot,timeline_sample_kv, get_plot_diff_data
from util.hedge_log import initlog
from util.recover_depth import recoveryDepth
from util.statistic_method_v2 import describe_series
from load_s3_data import *

def get_highest_frequency_data(begin_time, end_time, exchange, symbol, plot_interval_us=None, is_adj_time=True):
    '''
    根据交易所特性获取最高频的orderbook数据
    比如说 binance 为tick， OKEX为depth
    '''
        
    if exchange =='okex':
        highest_LOB = LoadS3Data.get_cex_depth(begin_time, end_time, exchange, symbol)
        depth_all = []
        for i in highest_LOB:
            bids = i['bids']
            asks = i['asks']
            time = i['time']
            depth_all.append([time]+[ i['p'] for i in bids ]+[ i['s'] for i in bids ]+[ i['p'] for i in asks ]+[ i['s'] for i in asks ])
            #break
        highest_LOB = pd.DataFrame(depth_all,columns=['time','bid1','bid2','bid3','bid4','bid5','bidqty1','bidqty2','bidqty3','bidqty4','bidqty5',
                                        'ask1','ask2','ask3','ask4','ask5','askqty1','askqty2','askqty3','askqty4','askqty5'])
        highest_LOB = highest_LOB[['time','bid1','bidqty1','ask1','askqty1']]
    #if exchange =='binance':
    else:
        highest_LOB = LoadS3Data.get_cex_ticker(begin_time, end_time,symbol,exchange,plot_interval_us=plot_interval_us, is_adj_time=is_adj_time)
        highest_LOB = pd.DataFrame(highest_LOB).rename(columns={'tp':'time','ap':'ask1','aa':'askqty1','bp':'bid1','ba':'bidqty1'})
        highest_LOB = highest_LOB[['time','ask1','askqty1','bid1','bidqty1']]
    return highest_LOB.rename(columns = {'bidqty1':'bid1_qty','askqty1':'ask1_qty'})


# load libraries 
import math
import decimal 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from scipy.linalg import block_diag
from scipy.linalg import eig

from StoikovEstimator import processing_data, estimate_transi_prob, f_cal_G6, get_default_K, estimate_old

In [3]:
exchange = 'binance'
symbol="eth_usdt"
begin_time = datetime.datetime(2024, 3, 10, 0, tzinfo=TZ_8)
end_time = begin_time + datetime.timedelta(days=1)

df_data = get_highest_frequency_data(begin_time, end_time, exchange, symbol, plot_interval_us=None, is_adj_time=True)

In [4]:
# load data
# df_data = pd.read_parquet('../Data/20240310/eth_usdt_binance_LOB.parquet')
df_data.columns = ['time_seconds', 'ask_price', 'ask_size', 'bid_price', 'bid_size']
# df_data['time_seconds'] = ((df_data.time_seconds)/1000 +1).astype(int)
# df_data = df_data.drop_duplicates(subset=['time_seconds'],keep='last').reset_index(drop=True)

test_percentage = 0.4
df_data_test = df_data.iloc[-int(test_percentage*len(df_data.time_seconds)):,].reset_index(drop=True)
df_data = df_data.iloc[:-len(df_data_test),].reset_index(drop=True)


# hyper-parameter
# number of imbalance 
n_imb = 11
# number of spread 
n_spread = 2
# delta t 
dt = 10

# grid of mid price change [half tick, one tick]
# K = np.array([-0.00001, -0.000005, 0.000005, 0.00001])
K = get_default_K(df_data)
print("K is", K)

K is [-0.01  -0.005  0.005  0.01 ]


### 使用过去的数据估计参数

In [5]:
# processing data 
df_data_to_use = processing_data(df_data, n_imb, n_spread, dt)

# estimation of transition probabiliies
G1,B,Q,Q2,R1,R2 = estimate_transi_prob(df_data_to_use, K, n_imb, n_spread)

# calculate micro price 
G6 = f_cal_G6(G1, B)
df_data_to_use

tick_size is 0.01


In [6]:
# plot 
imb=np.linspace(0,1,n_imb)
plt.figure(figsize=(13, 6), dpi=80)
for i in range(0,n_spread):
    plt.plot(imb,G6[(0+i*n_imb):(n_imb+i*n_imb)],label="spread = "+str(i+1))
# plt.ylim(-0.005,0.005)
plt.legend(loc='upper left')
plt.title('microprice adjustment')
plt.xlabel('Imbalance')

In [7]:
G6

### out of sample test   

In [ ]:
# processing data 
df_data_test_to_use = processing_data(df_data_test, n_imb, n_spread, dt, is_filter_symm=False)

# 记录需要的调整
spread1_adjustment = dict((i,G6[i]) for i in range(n_imb))
spread2_adjustment = dict((i,G6[i+11]) for i in range(n_imb))

df_data_test_to_use['micro_adj'] = df_data_test_to_use.loc[df_data_test_to_use.spread==1*K[3],].imb_bucket.map(spread1_adjustment)
df_data_test_to_use['micro_price'] = df_data_test_to_use['mid'] + df_data_test_to_use['micro_adj']
valid_test = df_data_test_to_use.loc[df_data_test_to_use.micro_price.notna(),]
valid_test = valid_test.drop_duplicates(subset=['time_seconds'],keep='last').reset_index(drop=True)

valid_test

In [ ]:
# 500ms 后的预测效果
valid_test['future_price'] = valid_test.mid.shift(-500)

score_mid = (valid_test.future_price - valid_test.mid).abs().mean()
score_wmid = (valid_test.future_price - valid_test.wmid).abs().mean()
score_micro_price = (valid_test.future_price - valid_test.micro_price).abs().mean()

print("mse of mid:", score_mid)
print("mse of wmid:", score_wmid)
print("mse of micro-price:", score_micro_price)

print('improve of wmid to mid:', score_mid - score_wmid)
print('improve of micro to wmid:', score_wmid - score_micro_price)

In [ ]:
# 1s 后的预测效果
valid_test['future_price'] = valid_test.mid.shift(-1000)

score_mid = (valid_test.future_price - valid_test.mid).abs().mean()
score_wmid = (valid_test.future_price - valid_test.wmid).abs().mean()
score_micro_price = (valid_test.future_price - valid_test.micro_price).abs().mean()

print("mse of mid:", score_mid)
print("mse of wmid:", score_wmid)
print("mse of micro-price:", score_micro_price)

print('improve of wmid to mid:', score_mid - score_wmid)
print('improve of micro to wmid:', score_wmid - score_micro_price)

In [ ]:
# 3s 后的预测效果
valid_test['future_price'] = valid_test.mid.shift(-3000)

score_mid = (valid_test.future_price - valid_test.mid).abs().mean()
score_wmid = (valid_test.future_price - valid_test.wmid).abs().mean()
score_micro_price = (valid_test.future_price - valid_test.micro_price).abs().mean()

print("mse of mid:", score_mid)
print("mse of wmid:", score_wmid)
print("mse of micro-price:", score_micro_price)

print('improve of wmid to mid:', score_mid - score_wmid)
print('improve of micro to wmid:', score_wmid - score_micro_price)